In [1]:
import pandas as pd

In [2]:
import numpy as np

### Carregar dados

In [3]:
jogos = pd.read_csv("data/jogos.csv")

In [4]:
jogos.head()

,Ano,Mandante,Visitante,Placar_mandante,Placar_visitante
0,2017,Vasco,Coritiba,1,1
1,2017,Flamengo,Bahia,4,1
2,2017,Sport Recife,Santos,1,1
3,2017,Palmeiras,Ponte Preta,2,0
4,2017,Vitória,Atlético-PR,2,3


In [5]:
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1431 entries, 0 to 1430
Data columns (total 5 columns):
Ano                 1431 non-null int64
Mandante            1431 non-null object
Visitante           1431 non-null object
Placar_mandante     1431 non-null int64
Placar_visitante    1431 non-null int64
dtypes: int64(3), object(2)
memory usage: 56.0+ KB


### Remover espaços em branco dos nomes dos times

In [6]:
jogos_obj = jogos.select_dtypes(['object'])
jogos[jogos_obj.columns] = jogos_obj.apply(lambda x: x.str.strip())

### Lista de times participantes

In [7]:
times = list(jogos["Visitante"].unique())

In [8]:
times.sort()

In [9]:
times

['América-MG',
 'Atlético-GO',
 'Atlético-MG',
 'Atlético-PR',
 'Avaí',
 'Bahia',
 'Botafogo',
 'Chapecoense',
 'Corinthians',
 'Coritiba',
 'Criciúma',
 'Cruzeiro',
 'Figueirense',
 'Flamengo',
 'Fluminense',
 'Goias',
 'Grêmio',
 'Internacional',
 'Joinville',
 'Palmeiras',
 'Ponte Preta',
 'Santa Cruz',
 'Santos',
 'Sport Recife',
 'São Paulo',
 'Vasco',
 'Vitória']

In [10]:
len(times)

27

### Criar coluna com resultado

In [11]:
def resultado(row):
    if (row["Placar_mandante"] > row["Placar_visitante"]):
        return "MANDANTE"
    elif (row["Placar_mandante"] < row["Placar_visitante"]):
        return "VISITANTE"
    else:
        return "EMPATE"

In [12]:
jogos['Resultado'] = jogos.apply (lambda row: resultado(row),axis=1)

In [13]:
jogos.head()

,Ano,Mandante,Visitante,Placar_mandante,Placar_visitante,Resultado
0,2017,Vasco,Coritiba,1,1,EMPATE
1,2017,Flamengo,Bahia,4,1,MANDANTE
2,2017,Sport Recife,Santos,1,1,EMPATE
3,2017,Palmeiras,Ponte Preta,2,0,MANDANTE
4,2017,Vitória,Atlético-PR,2,3,VISITANTE


### Totais de resultados por ano

In [25]:
jogos.groupby(["Ano", "Resultado"])[["Ano"]].count()

Ano
Ano  Resultado     
2014 EMPATE      92
     MANDANTE   197
     VISITANTE   91
2015 EMPATE      91
     MANDANTE   200
     VISITANTE   89
2016 EMPATE      95
     MANDANTE   202
     VISITANTE   83
2017 EMPATE      77
     MANDANTE   124
     VISITANTE   90

### Totais por resultado

In [28]:
totais_resultados = jogos.groupby("Resultado").size()

In [29]:
totais_resultados

Resultado
EMPATE       355
MANDANTE     723
VISITANTE    353
dtype: int64

In [39]:
total_jogos = totais_resultados.sum()

In [40]:
totais_resultados.keys()

Index(['EMPATE', 'MANDANTE', 'VISITANTE'], dtype='object', name='Resultado')

In [41]:
totais_resultados.values

array([355, 723, 353])

In [42]:
resultados_DF = pd.DataFrame({'Resultados': totais_resultados.keys(), 'Totais': totais_resultados.values})

In [43]:
resultados_DF

,Resultados,Totais
0,EMPATE,355
1,MANDANTE,723
2,VISITANTE,353


In [47]:
def percentual(valor, total):
    return round(valor/total, 3)

In [48]:
resultados_DF['Perc'] = resultados_DF.apply (lambda row: percentual(row["Totais"], total_jogos),axis=1)

In [49]:
resultados_DF

,Resultados,Totais,Perc
0,EMPATE,355,0.248
1,MANDANTE,723,0.505
2,VISITANTE,353,0.247


### Calcular resultados de um time

In [57]:
len(jogos[(jogos["Mandante"] == "São Paulo") & (jogos["Resultado"] == "MANDANTE")]) 

39

In [58]:
len(jogos[(jogos["Mandante"] == "São Paulo") & (jogos["Resultado"] == "EMPATE")])

22

In [59]:
len(jogos[(jogos["Mandante"] == "São Paulo") & (jogos["Resultado"] == "VISITANTE")])

10

In [60]:
len(jogos[(jogos["Visitante"] == "São Paulo") & (jogos["Resultado"] == "MANDANTE")]) 

37

In [61]:
len(jogos[(jogos["Visitante"] == "São Paulo") & (jogos["Resultado"] == "EMPATE")]) 

13

In [62]:
len(jogos[(jogos["Visitante"] == "São Paulo") & (jogos["Resultado"] == "VISITANTE")]) 

22

In [98]:
def resultadosTime(time, casa, resultado):
    return len(jogos[(jogos[casa] == time) & (jogos["Resultado"] == resultado)]) 

In [99]:
resultadosTime("São Paulo", "Mandante", "MANDANTE")

39

In [100]:
resultados_time_DF = pd.DataFrame({'Times': times })

In [101]:
resultados_time_DF["vitorias_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Mandante", "MANDANTE"),axis=1)
resultados_time_DF["empates_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Mandante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Mandante", "VISITANTE"),axis=1)
resultados_time_DF["vitorias_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Visitante", "VISITANTE"),axis=1)
resultados_time_DF["empates_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Visitante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Visitante", "MANDANTE"),axis=1)

In [102]:
resultados_time_DF.head(3)

,Times,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora
0,América-MG,7,3,9,0,4,15
1,Atlético-GO,3,3,9,4,2,8
2,Atlético-MG,42,13,17,23,24,24


In [103]:
def totalJogos(row):
    return row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"] + row["vitorias_fora"] + row["empates_fora"] +  row["derrotas_fora"]

In [104]:
resultados_time_DF["TOTAL"] = resultados_time_DF.apply (lambda row: totalJogos(row),axis=1)

In [105]:
resultados_time_DF.head(3)

,Times,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,TOTAL
0,América-MG,7,3,9,0,4,15,38
1,Atlético-GO,3,3,9,4,2,8,29
2,Atlético-MG,42,13,17,23,24,24,143


In [106]:
def aproveitamentoMandante(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    return (3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante)

In [107]:
resultados_time_DF["aprov_casa"] = resultados_time_DF.apply (lambda row: aproveitamentoMandante(row),axis=1)
resultados_time_DF.head(3)


,Times,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,TOTAL,aprov_casa
0,América-MG,7,3,9,0,4,15,38,0.421053
1,Atlético-GO,3,3,9,4,2,8,29,0.266667
2,Atlético-MG,42,13,17,23,24,24,143,0.643519


In [108]:
def aproveitamentoVisitante(row):
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"]) / (3*total_jogos_visitante)

In [109]:
resultados_time_DF["aprov_fora"] = resultados_time_DF.apply (lambda row: aproveitamentoVisitante(row),axis=1)
resultados_time_DF.head(3)

,Times,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,TOTAL,aprov_casa,aprov_fora
0,América-MG,7,3,9,0,4,15,38,0.421053,0.070175
1,Atlético-GO,3,3,9,4,2,8,29,0.266667,0.333333
2,Atlético-MG,42,13,17,23,24,24,143,0.643519,0.436620


In [110]:
def aproveitamentoTotal(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"] + 3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante + 3*total_jogos_visitante)

In [128]:
resultados_time_DF["aprov_total"] = resultados_time_DF.apply (lambda row: aproveitamentoTotal(row),axis=1)
resultados_time_DF.sort_values(["aprov_total"], ascending=[0]).head(3)

,Times,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,TOTAL,aprov_casa,aprov_fora,aprov_total
8,Corinthians,47,19,6,28,20,23,143,0.740741,0.488263,0.615385
11,Cruzeiro,39,21,11,27,14,31,143,0.647887,0.439815,0.543124
22,Santos,49,11,11,17,23,32,143,0.741784,0.342593,0.540793


In [136]:
resultados_time_DF[["Times", "TOTAL", "aprov_casa", "aprov_fora", "aprov_total"]].sort_values(["aprov_total"], ascending=[0])

,Times,TOTAL,aprov_casa,aprov_fora,aprov_total
8,Corinthians,143,0.740741,0.488263,0.615385
11,Cruzeiro,143,0.647887,0.439815,0.543124
22,Santos,143,0.741784,0.342593,0.540793
2,Atlético-MG,143,0.643519,0.436620,0.540793
16,Grêmio,143,0.694836,0.388889,0.540793
19,Palmeiras,143,0.606481,0.431925,0.519814
13,Flamengo,143,0.611111,0.403756,0.508159
24,São Paulo,143,0.652582,0.365741,0.508159
17,Internacional,113,0.708333,0.292398,0.498525
3,Atlético-PR,143,0.633803,0.300926,0.466200
